Late time kilonova

In this code we will post-processes the results for NSM simulations perform by Winnet.

-Libraries

In [2]:
#libraries

#Data
import pandas as pd
#import h5py

#Plots
import matplotlib.pyplot as plt

#fits
#from scipy.optimize import curve_fit

#math
import numpy as np

#Constants
#from scipy.constants import speed_of_light as c
#from scipy.constants import Avogadro as N_A
#from scipy.constants import Boltzmann as kb
#from scipy.constants import Stefan_Boltzmann as Sb

#Usefull
from tqdm.notebook import tqdm
import os
from scipy.interpolate import interp1d
#%matplotlib widget

-Reading data without HDF5 format

In [ ]:


folder = 'Example_NSM_dyn_ejecta_rosswog_varios_t_sinalpha\snaps'
files = os.listdir(folder)

abundances_winnet=[]
time_Winnet=[]

for f in tqdm(files):

    abundances_winnet.append(np.loadtxt(os.path.join(folder, f), skiprows=3))
    time_Winnet.append(float(np.loadtxt(os.path.join(folder, f), skiprows=1,max_rows=1, usecols=0)))

abundances_winnet = np.array(abundances_winnet)
time_Winnet = np.array(time_Winnet) 

  0%|          | 0/554 [00:00<?, ?it/s]

-Interpolation

In [3]:
initial_time=1 #seconds
final_time=24*60*60 #seconds
N_steps=10000
time = np.exp(np.linspace(np.log(initial_time), np.log(final_time), N_steps))
abundances_time= np.empty((abundances_winnet.shape[1],N_steps))
for i in tqdm(range(abundances_winnet.shape[1])):
    spline_interp = interp1d(time_Winnet, abundances_winnet[:,i,3], kind='linear')
    abundances_time[i] = spline_interp(time)

  0%|          | 0/6757 [00:00<?, ?it/s]

-Funtion that finds the nuclei and return X_i(t)

In [ ]:

def abundance_time_of_nuclei_n_z(n,z):
    
    idx_start = np.searchsorted(abundances_winnet[0,:,1], z, side='left')
    idx_end = np.searchsorted(abundances_winnet[0,:,1], z, side='right')
    if idx_end>idx_start:
        j=np.searchsorted(abundances_winnet[0,:,0][idx_start:idx_end],n)+idx_start
        if abundances_winnet[0,:,0][j]==n and abundances_winnet[0,:,1][j]==z:
            return abundances_time[j]
        else:
            return np.zeros(N_steps)
    else:
        return np.zeros(N_steps)

-Masses of nuclei

In [ ]:
file_masses="/mnt/c/users/Diego_Hernandez/winnet/WinNet-master/WinNet-master/data/winvne_v2.0.dat"